# Ethereum Price Forecasting with Monte Carlo Simulation

## 🎯 Objective
This notebook provides a quantitative framework for forecasting Ethereum (ETH-USD) price movements using Monte Carlo simulation. The primary goal is to derive actionable insights for trading short-term crypto options (1-3 week expiries) by modeling the probability distribution of future prices.

## 📈 Methodology
1.  **Data Collection**: Fetch historical daily price data for ETH-USD from Yahoo Finance.
2.  **Parameter Estimation**: Calculate the historical daily log returns to estimate the asset's drift (mean return) and volatility (standard deviation of returns).
3.  **Stochastic Modeling**: Employ Geometric Brownian Motion (GBM) to simulate thousands of potential future price paths. GBM is a standard model for financial assets, assuming that price returns are normally distributed.
4.  **Probabilistic Analysis**: Analyze the distribution of simulated prices at specific time horizons (e.g., 7, 14, and 21 days) to quantify risk and opportunity.
5.  **Options Strategy Insights**: Translate the simulation results into practical insights for options trading, such as:
    *   Probability of expiring in-the-money for various strike prices.
    *   Expected price range over the forecast period.
    *   Identifying potential mispricings in options based on model-derived probabilities versus market-implied volatility.

This analysis is designed to be a powerful tool for any trader looking to apply rigorous quantitative methods to the volatile cryptocurrency markets.

In [8]:
# Core Libraries
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('dark_background')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Libraries imported successfully.")

✅ Libraries imported successfully.


## 1. Data Collection & Preparation

First, we fetch the historical daily price data for ETH-USD. We will use a sufficiently long lookback period to capture various market regimes and accurately estimate historical volatility and drift.

In [9]:
# --- Parameters ---
TICKER = 'ETH-USD'
START_DATE = '2017-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

# --- Data Fetching ---
print(f"📊 Fetching {TICKER} data from {START_DATE} to {END_DATE}...")
try:
    eth_data = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)
    
    if eth_data.empty:
        raise ValueError("No data fetched. Check ticker or date range.")
    
    # Calculate log returns for modeling
    eth_data['Log_Return'] = np.log(eth_data['Close'] / eth_data['Close'].shift(1))
    eth_data = eth_data.dropna()
    
    print(f"✅ Data fetched and prepared successfully:")
    print(f"   - Date range: {eth_data.index.min().strftime('%Y-%m-%d')} to {eth_data.index.max().strftime('%Y-%m-%d')}")
    print(f"   - Total observations: {len(eth_data):,}")
    
    print("\n📋 Sample Data with Log Returns:")
    display(eth_data.head())

    # Plot historical prices
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=eth_data.index, y=eth_data['Close'], mode='lines', name='ETH Close Price'))
    fig.update_layout(
        title=f'{TICKER} Historical Closing Price',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_dark'
    )
    fig.show()
    
except Exception as e:
    print(f"❌ Error fetching data: {e}")

📊 Fetching ETH-USD data from 2017-01-01 to 2025-07-21...
✅ Data fetched and prepared successfully:
   - Date range: 2017-11-10 to 2025-07-20
   - Total observations: 2,810

📋 Sample Data with Log Returns:


Price,Close,High,Low,Open,Volume,Log_Return
Ticker,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD,
Date,,,,,,
2017-11-10,299.252991,324.717987,294.541992,320.670990,885985984,-0.069790
2017-11-11,314.681000,319.453003,298.191986,298.585999,842300992,0.050270
2017-11-12,307.907990,319.153015,298.513000,314.690002,1613479936,-0.021758
2017-11-13,316.716003,328.415009,307.024994,307.024994,1041889984,0.028204
2017-11-14,337.631012,340.177002,316.763000,316.763000,1069680000,0.063948


## 2. Monte Carlo Simulation Engine

Here, we build the core of our analysis. The simulation engine uses the calculated historical drift and volatility to generate a large number of possible price paths according to the Geometric Brownian Motion formula:

$ S_{t} = S_{t-1} \times e^{((	\mu - \frac{1}{2}\sigma^2) \Delta t + \sigma \sqrt{\Delta t} Z)} $

Where:
- $S_t$ is the price at time t
- $\mu$ is the drift (mean of log returns)
- $\sigma$ is the volatility (standard deviation of log returns)
- $\Delta t$ is the time step (1 day)
- $Z$ is a random number from the standard normal distribution

This allows us to create a statistically robust distribution of potential future outcomes.

## 2.5. Is the Assumption of Normality Correct? (A Nod to Mandelbrot & Taleb)

As you astutely pointed out, the Geometric Brownian Motion model is built on the assumption that log returns are normally distributed. However, financial markets often exhibit characteristics that violate this assumption, such as:
-   **Fat Tails (Leptokurtosis)**: Extreme price movements (both positive and negative) occur more frequently than a normal distribution would predict.
-   **Skewness**: The distribution is not symmetric; there might be a tendency for larger negative returns than positive ones, or vice-versa.

Let's investigate the historical log returns of ETH-USD to see if they conform to a normal distribution. We'll use statistical tests and visualizations to check.

In [10]:
# Interpretation
print("\n--- Interpretation ---")
print("""
- **Skewness**: Measures the asymmetry of the distribution.
    - **0**: Perfectly symmetrical (like a normal distribution).
    - **> 0 (Positive Skew)**: The right tail is longer. There are more small losses and a few large gains.
    - **< 0 (Negative Skew)**: The left tail is longer. There are more small gains and a few large losses. A value of -1 indicates a significant negative skew.
- **Kurtosis**: Measures the "tailedness" of the distribution.
    - **0**: Has the same tailedness as a normal distribution.
    - **> 0 (Positive Kurtosis / Leptokurtic)**: "Fat tails". Extreme outcomes (both positive and negative) are more likely than in a normal distribution. A value of 1 or more is significant.
    - **< 0 (Negative Kurtosis / Platykurtic)**: "Thin tails". Extreme outcomes are less likely.
""")
if p_value < 0.05:
    print("🚨 The p-value is less than 0.05. We REJECT the null hypothesis that the returns are normally distributed.")
    print("   - The positive kurtosis indicates 'fat tails' (leptokurtosis), meaning extreme events are more likely than a normal distribution predicts.")
else:
    print("✅ The p-value is greater than 0.05. We FAIL to reject the null hypothesis. The data appears to be normally distributed.")


--- Interpretation ---

- **Skewness**: Measures the asymmetry of the distribution.
    - **0**: Perfectly symmetrical (like a normal distribution).
    - **> 0 (Positive Skew)**: The right tail is longer. There are more small losses and a few large gains.
    - **< 0 (Negative Skew)**: The left tail is longer. There are more small gains and a few large losses. A value of -1 indicates a significant negative skew.
- **Kurtosis**: Measures the "tailedness" of the distribution.
    - **0**: Has the same tailedness as a normal distribution.
    - **> 0 (Positive Kurtosis / Leptokurtic)**: "Fat tails". Extreme outcomes (both positive and negative) are more likely than in a normal distribution. A value of 1 or more is significant.
    - **< 0 (Negative Kurtosis / Platykurtic)**: "Thin tails". Extreme outcomes are less likely.

🚨 The p-value is less than 0.05. We REJECT the null hypothesis that the returns are normally distributed.
   - The positive kurtosis indicates 'fat tails' (leptokurto

### Using a More Realistic Distribution: The Student's t-Distribution

Given the evidence of fat tails, a more robust approach is to replace the normal distribution in our Monte Carlo simulation with a **Student's t-distribution**. The t-distribution has an additional parameter, the **degrees of freedom (df)**, which controls the "thickness" of the tails. A lower `df` value results in fatter tails, making it better suited for modeling financial returns.

We will:
1.  Fit a t-distribution to our historical log returns to find the optimal parameters (`df`, `loc`, `scale`).
2.  Create a new Monte Carlo simulation engine that samples from this fitted t-distribution instead of the normal distribution.
3.  Re-run our analysis to see how the forecasts change.

In [11]:
def visualize_simulation_paths(paths, start_price, forecast_days, title_suffix=""):
    """
    Visualizes the Monte Carlo simulation results with a probability cone.
    """
    # Create a date range for the x-axis
    last_date = eth_data.index[-1]
    forecast_dates = pd.to_datetime([last_date + timedelta(days=i) for i in range(forecast_days + 1)])
    
    # Calculate percentiles for the probability cone
    p5 = np.percentile(paths, 5, axis=1)
    p25 = np.percentile(paths, 25, axis=1)
    p50 = np.percentile(paths, 50, axis=1) # Median
    p75 = np.percentile(paths, 75, axis=1)
    p95 = np.percentile(paths, 95, axis=1)
    
    fig = go.Figure()

    # Add percentile bands (the "cone")
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p95, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'), name='95th Percentile'
    ))
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p5, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'),
        fill='tonexty', fillcolor='rgba(173, 216, 230, 0.2)', name='90% Confidence Interval'
    ))
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p75, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'), name='75th Percentile'
    ))
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p25, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'),
        fill='tonexty', fillcolor='rgba(173, 216, 230, 0.4)', name='50% Confidence Interval'
    ))

    # Add the median path
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p50, mode='lines', line=dict(color='white', dash='dash'), name='Median Path'
    ))
    
    # Add a sample of simulated paths
    num_paths_to_plot = 100
    for i in range(num_paths_to_plot):
        fig.add_trace(go.Scatter(
            x=forecast_dates, y=paths[:, i], mode='lines',
            line=dict(width=0.5, color='rgba(255, 255, 255, 0.1)'),
            showlegend=False
        ))

    fig.update_layout(
        title=f'Monte Carlo Simulation: {SIMULATIONS} Paths ({title_suffix})',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        showlegend=True
    )
    fig.show()

# --- Run New Simulation with t-distribution ---
print("\n⚙️ Running new simulation with Student's t-distribution...")
price_paths_t = run_monte_carlo_t_dist(START_PRICE, df, loc, scale, FORECAST_HORIZON_DAYS, SIMULATIONS)
print(f"✅ t-distribution simulation complete. Generated {price_paths_t.shape[1]:,} price paths.")

# --- Visualize the new paths ---
visualize_simulation_paths(price_paths_t, START_PRICE, FORECAST_HORIZON_DAYS, title_suffix="Student's t-Distribution")


⚙️ Running new simulation with Student's t-distribution...
✅ t-distribution simulation complete. Generated 10,000 price paths.


In [12]:
def run_monte_carlo_simulation(start_price, drift, volatility, days, simulations):
    """
    Runs a Monte Carlo simulation for asset prices using Geometric Brownian Motion.
    
    Args:
        start_price (float): The initial price of the asset.
        drift (float): The historical mean of daily log returns.
        volatility (float): The historical standard deviation of daily log returns.
        days (int): The number of days to simulate into the future.
        simulations (int): The number of simulation paths to generate.
        
    Returns:
        numpy.ndarray: A 2D array of simulated price paths.
    """
    # Pre-calculate constants for efficiency
    dt = 1
    shock = (drift - 0.5 * volatility**2) * dt
    drift_component = shock
    volatility_component = volatility * np.sqrt(dt)
    
    # Generate random shocks
    random_shocks = np.random.normal(0, 1, (days, simulations))
    
    # Calculate daily returns
    daily_returns = np.exp(drift_component + volatility_component * random_shocks)
    
    # Create price paths
    price_paths = np.zeros((days + 1, simulations))
    price_paths[0] = start_price
    
    for t in range(1, days + 1):
        price_paths[t] = price_paths[t - 1] * daily_returns[t - 1]
        
    return price_paths

# --- Estimate Model Parameters ---
# Use the last 365 days for more recent market dynamics
recent_data = eth_data.tail(365)
mu = recent_data['Log_Return'].mean()
sigma = recent_data['Log_Return'].std()

# Annualized Volatility
annualized_volatility = sigma * np.sqrt(365)

print("📈 Model Parameters (based on last 365 days):")
print(f"   - Daily Drift (μ): {mu:.5f}")
print(f"   - Daily Volatility (σ): {sigma:.5f}")
print(f"   - Annualized Volatility: {annualized_volatility:.2%}")

# --- Simulation Parameters ---
SIMULATIONS = 10000
FORECAST_HORIZON_DAYS = 21  # 3 weeks for options trading
START_PRICE = eth_data['Close'].iloc[-1]

print("\n⚙️ Simulation Setup:")
print(f"   - Start Price: ${START_PRICE:,.2f}")
print(f"   - Number of Simulations: {SIMULATIONS:,}")
print(f"   - Forecast Horizon: {FORECAST_HORIZON_DAYS} days")

# --- Run Simulation ---
price_paths = run_monte_carlo_simulation(START_PRICE, mu, sigma, FORECAST_HORIZON_DAYS, SIMULATIONS)

print(f"\n✅ Monte Carlo simulation complete. Generated {price_paths.shape[1]:,} price paths for {price_paths.shape[0]-1} days.")

📈 Model Parameters (based on last 365 days):
   - Daily Drift (μ): 0.00018
   - Daily Volatility (σ): 0.03889
   - Annualized Volatility: 74.30%

⚙️ Simulation Setup:


TypeError: unsupported format string passed to Series.__format__

## 3. Visualizing the Simulated Price Paths

Visualizing the simulated paths helps in understanding the range of potential outcomes. We plot a sample of the simulations along with the median and key percentile ranges to create a "probability cone." This cone represents the most likely area where the price will trade over the forecast horizon.

In [13]:
def visualize_simulation_paths(paths, start_price, forecast_days):
    """
    Visualizes the Monte Carlo simulation results with a probability cone.
    """
    # Create a date range for the x-axis
    last_date = eth_data.index[-1]
    forecast_dates = pd.to_datetime([last_date + timedelta(days=i) for i in range(forecast_days + 1)])
    
    # Calculate percentiles for the probability cone
    p5 = np.percentile(paths, 5, axis=1)
    p25 = np.percentile(paths, 25, axis=1)
    p50 = np.percentile(paths, 50, axis=1) # Median
    p75 = np.percentile(paths, 75, axis=1)
    p95 = np.percentile(paths, 95, axis=1)
    
    fig = go.Figure()

    # Add percentile bands (the "cone")
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p95, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'), name='95th Percentile'
    ))
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p5, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'),
        fill='tonexty', fillcolor='rgba(173, 216, 230, 0.2)', name='90% Confidence Interval'
    ))
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p75, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'), name='75th Percentile'
    ))
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p25, mode='lines', line=dict(color='rgba(173, 216, 230, 0)'),
        fill='tonexty', fillcolor='rgba(173, 216, 230, 0.4)', name='50% Confidence Interval'
    ))

    # Add the median path
    fig.add_trace(go.Scatter(
        x=forecast_dates, y=p50, mode='lines', line=dict(color='white', dash='dash'), name='Median Path'
    ))
    
    # Add a sample of simulated paths
    num_paths_to_plot = 100
    for i in range(num_paths_to_plot):
        fig.add_trace(go.Scatter(
            x=forecast_dates, y=paths[:, i], mode='lines',
            line=dict(width=0.5, color='rgba(255, 255, 255, 0.1)'),
            showlegend=False
        ))

    fig.update_layout(
        title=f'Monte Carlo Simulation: {SIMULATIONS} Potential Price Paths for ETH-USD ({FORECAST_HORIZON_DAYS} Days)',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        showlegend=True
    )
    fig.show()

visualize_simulation_paths(price_paths, START_PRICE, FORECAST_HORIZON_DAYS)

NameError: name 'price_paths' is not defined

## 4. Analysis of Terminal Prices & Options Insights

This is the most critical part for a trader. We analyze the distribution of the final prices from our simulation to derive concrete probabilities. This information can be directly used to evaluate options contracts.

### Key Questions for an Options Trader:
1.  What is the expected price at expiration?
2.  What is the probability that the price will finish above a certain call strike or below a certain put strike?
3.  What is the likely range of outcomes, and how does this compare to the market's pricing of options (implied volatility)?

In [ ]:
def analyze_terminal_prices(paths, start_price, forecast_days):
    """
    Analyzes and visualizes the distribution of terminal prices from the simulation.
    """
    terminal_prices = paths[-1, :]
    
    # --- Plot Distribution ---
    fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], subplot_titles=("Distribution of Terminal Prices", "Key Statistics"))
    
    # Histogram
    fig.add_trace(go.Histogram(x=terminal_prices, nbinsx=100, name='Frequency', marker_color='#1f77b4'), row=1, col=1)
    
    # Add vertical lines for key stats
    mean_price = terminal_prices.mean()
    median_price = np.median(terminal_prices)
    p5 = np.percentile(terminal_prices, 5)
    p95 = np.percentile(terminal_prices, 95)
    
    fig.add_vline(x=start_price, line=dict(color='white', dash='dash'), name='Start Price', row=1, col=1)
    fig.add_vline(x=mean_price, line=dict(color='yellow', dash='dash'), name='Mean Price', row=1, col=1)
    fig.add_vline(x=median_price, line=dict(color='cyan', dash='dash'), name='Median Price', row=1, col=1)
    
    fig.update_layout(
        title_text=f'Analysis of ETH-USD Price After {forecast_days} Days',
        template='plotly_dark',
        showlegend=False
    )
    
    # --- Display Statistics ---
    stats_text = (
        f"<b>Start Price:</b> ${start_price:,.2f}<br>"
        f"<b>Mean Forecast:</b> ${mean_price:,.2f}<br>"
        f"<b>Median Forecast:</b> ${median_price:,.2f}<br>"
        f"<br><b>Confidence Intervals:</b><br>"
        f"<b>5th Percentile:</b> ${p5:,.2f}<br>"
        f"<b>95th Percentile:</b> ${p95:,.2f}<br>"
        f"(90% chance price is between these values)"
    )
    
    fig.add_trace(go.Indicator(
        mode = "number",
        value = median_price,
        title = {"text": "Median Forecast"},
        domain = {'row': 0, 'column': 1}
    ))

    fig.show()
    
    # --- Probability Analysis for Options Trading ---
    print("📈 Options Trading Insights:")
    
    strike_levels = [-0.20, -0.10, -0.05, 0.05, 0.10, 0.20] # % change from start price
    results = []
    
    for level in strike_levels:
        strike_price = start_price * (1 + level)
        
        if level > 0: # For Call Options
            prob = (terminal_prices > strike_price).mean()
            results.append({
                "Option Type": "Call",
                "Strike Price": f"${strike_price:,.2f} ({level:+.0%})",
                "Prob. of Finishing ITM": f"{prob:.2%}"
            })
        else: # For Put Options
            prob = (terminal_prices < strike_price).mean()
            results.append({
                "Option Type": "Put",
                "Strike Price": f"${strike_price:,.2f} ({level:+.0%})",
                "Prob. of Finishing ITM": f"{prob:.2%}"
            })
            
    results_df = pd.DataFrame(results)
    display(results_df)
    
    return results_df

# Run the analysis for different time horizons relevant to options
print("="*50)
print("ANALYSIS FOR 7-DAY EXPIRATION")
print("="*50)
paths_7d = run_monte_carlo_simulation(START_PRICE, mu, sigma, 7, SIMULATIONS)
analysis_7d = analyze_terminal_prices(paths_7d, START_PRICE, 7)

print("\n" + "="*50)
print("ANALYSIS FOR 14-DAY EXPIRATION")
print("="*50)
paths_14d = run_monte_carlo_simulation(START_PRICE, mu, sigma, 14, SIMULATIONS)
analysis_14d = analyze_terminal_prices(paths_14d, START_PRICE, 14)

print("\n" + "="*50)
print("ANALYSIS FOR 21-DAY EXPIRATION")
print("="*50)
paths_21d = run_monte_carlo_simulation(START_PRICE, mu, sigma, 21, SIMULATIONS)
analysis_21d = analyze_terminal_prices(paths_21d, START_PRICE, 21)

## 5. Summary & How to Use This Analysis

### 📝 Summary of Findings
This notebook executed a robust Monte Carlo simulation to forecast the price of Ethereum over 7, 14, and 21-day horizons. We have generated key outputs for a quantitative trader:
1.  **Probability Cones**: Visualizations of the likely price range over time.
2.  **Terminal Price Distributions**: Histograms showing the probability distribution of the price at each expiration horizon.
3.  **Probabilistic Forecasts**: A table quantifying the probability of the price finishing above/below various strike prices.

### 🚀 How to Use for Options Trading
The insights from this model can be a powerful component of an options trading strategy.

1.  **Identifying Mispriced Options**:
    *   The "Probability of Finishing ITM" (In-The-Money) is the model's core output.
    *   Compare this probability to the market's implied probability. An option's **Delta** is often used as a rough proxy for the market's perceived probability of expiring ITM.
    *   **Example**: If our model shows a 40% probability of a call option finishing ITM, but its Delta is only 0.25 (implying a ~25% chance), the model suggests the option may be **undervalued**. This could be a buying opportunity. Conversely, if the model shows a 15% probability and the Delta is 0.30, the option may be **overvalued**, suggesting a selling or shorting opportunity.

2.  **Structuring Trades**:
    *   **Buying Straddles/Strangles**: If the analysis shows a high probability of a large move in either direction (e.g., high probability of being outside the +/- 10% range), but options are cheap (low implied volatility), a long volatility strategy like a straddle could be profitable.
    *   **Selling Iron Condors**: If the model suggests a high probability of the price staying within a tight range (e.g., the 50% confidence interval is narrow), an iron condor or credit spread strategy could be attractive, especially if implied volatility is high.

3.  **Risk Management**:
    *   Use the 90% confidence interval as a guide for setting realistic profit targets and stop-losses.
    *   Understand the tail risk by looking at the 5th and 95th percentile outcomes.

### ⚠️ Important Caveats
- **Model Assumptions**: Geometric Brownian Motion assumes returns are normally distributed and independent, which may not hold true during extreme market events (fat tails).
- **Historical Data**: The model is based on historical volatility and drift, which are not guaranteed to persist in the future.
- **No Guarantees**: This is a probabilistic model, not a crystal ball. It provides a statistical edge, not certainty.

This framework should be used as one tool among many in a comprehensive trading system.